<a href="https://colab.research.google.com/github/diego690/AppSchool/blob/main/TensorFLowPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##importamos librerias de tensorflow 
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#descargamos el set de datos que vamos a utilizar como ejemplo que seria perros y gatos
#por las fallas que presenta estos meses
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
datos, metadatos = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)

In [ ]:
#visualizamos los datos
metadatos

In [ ]:
#si queremos mostrar 5 ejemplos del set de datos
tfds.as_dataframe(datos['train'].take(5),metadatos)

In [ ]:
#ahora si queremos algo mas "bonito"
tfds.show_examples(datos['train'].take(5),metadatos)

In [ ]:
#ahora si queremos un control total de lo que queremos ver tenemos:
import matplotlib.pyplot as plt
#ahora si queremos que todas las imagenes tenga una sola dimension 
import cv2
#agregamos un iteracion para las imagenes segun queramos visualizarlas

#primero si queremos que las imagenes tenga un tamaño general 
plt.figure(figsize=(20,20))

#tamañno de la imagen que luego se redimensionara
TAMANO_IMG = 100

#un for para recorrer hasta donde queramos mostrar
for i, (imagen, etiqueta) in enumerate(datos['train'].take(2)):
  imagen = cv2.resize(imagen.numpy(),(TAMANO_IMG,TAMANO_IMG))#redimensiono las imagenes
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)#cambiamos el color a b/n para mayor rapidez
  plt.subplot(5,5, i+1) #tamaño de la matriz si queremos usar mas de 5 imagenes
  plt.xticks([])
  plt.yticks([])
  plt.imshow(imagen,cmap='gray') #mostramos las imagenes

In [6]:
datos_entrenamiento = []

In [7]:
#para todos los datos de entranamiento
for i, (imagen,etiqueta) in enumerate(datos['train']):
  imagen = cv2.resize(imagen.numpy(),(TAMANO_IMG,TAMANO_IMG))
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)#cambiamos el color a b/n para mayor rapidez
  #para las imagenes modificadas junto con el cambio de forma y junto con el solo canal de color 
  imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG,1)#cambiar tamaño 100,100,1
  datos_entrenamiento.append([imagen,etiqueta])

In [ ]:
datos_entrenamiento[0] #primer indice, donde los valores de los pixele mostramos, y el numpy que es 1 para perro y 0 para gato

In [ ]:
len(datos_entrenamiento)#imagenes en el arreglo

In [9]:
#ahora preparamos los datos para el entramiento 
#creo dos variables x, y
X = [] #paa las imagens de entrada, es decir, pixeles
y = [] #para las etiquetas (perro o gato)

for imagen, etiqueta in datos_entrenamiento:
  X.append(imagen)
  y.append(etiqueta)

In [ ]:
X

In [10]:
#ahora normalizamos los datos
import numpy as np

X = np.array(X).astype(float)/255

In [11]:
y = np.array(y) #etiquetas en un arreglo simple

In [ ]:
y

In [ ]:
X.shape#info del arreglo

In [14]:
#probamos modelo denso
modeloDenso = tf.keras.models.Sequential([
                                          tf.keras.layers.Flatten(input_shape=(100,100,1)),
                                          tf.keras.layers.Dense(150,activation='relu'),#con 150 neuronas 
                                          tf.keras.layers.Dense(150,activation='relu'),
                                          tf.keras.layers.Dense(1,activation='sigmoid')#una neurona de salida con activacion para decir si es perrro o gato

])
#modelo convulocional, con 3 capas convulucionales y agrupacion maxima 32,64,128 filtros
modeloCNN = tf.keras.models.Sequential([
                                       tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
                                        tf.keras.layers.MaxPool2D(2,2),
                                         tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                        tf.keras.layers.MaxPool2D(2,2),
                                         tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
                                        tf.keras.layers.MaxPool2D(2,2),

                                        #con una capa densa de 100 neuronas con
                                        tf.keras.layers.Flatten(),
                                        #con una salida con sigmoid
                                        tf.keras.layers.Dense(100,activation='relu'),
                                        tf.keras.layers.Dense(1,activation='sigmoid')
])

#modelo cnn2
modeloCNN2 = tf.keras.models.Sequential([
                                       tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
                                        tf.keras.layers.MaxPool2D(2,2),
                                         tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                        tf.keras.layers.MaxPool2D(2,2),
                                         tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
                                        tf.keras.layers.MaxPool2D(2,2),

                                        #con un dropout de 1/2 antes de la capa densa de 250 neuronas
                                        tf.keras.layers.Dropout(0.5),
                                        tf.keras.layers.Flatten(),
                                        #con una salida con sigmoid
                                        tf.keras.layers.Dense(250,activation='relu'),
                                        tf.keras.layers.Dense(1,activation='sigmoid')
])

In [15]:
#para compilar los modelos se puede utilizar en este caso que solo tenemos 2 objetos, el crossentropy
modeloDenso.compile(optimizer = 'adam',
                    loss = 'binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN.compile(optimizer = 'adam',
                    loss = 'binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN2.compile(optimizer = 'adam',
                    loss = 'binary_crossentropy',
                    metrics=['accuracy'])

In [16]:
#utilizamos la siguiente libreria para saber como estan funcionando los modelos 
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
from gc import callbacks
#creamos una carpeta para cada modelo
tensorboardDenso = TensorBoard(log_dir='logs/denso')
#con fit le mando la X(imagenes) y las Y(etiquetas) y el lote, dejando un % para pruebas
modeloDenso.fit(X,y,batch_size=32,
                validation_split = 0.15,
                epochs = 100,
                callbacks=[tensorboardDenso])# por cada epoca que recorra y al final guarda en un archivo 



In [ ]:
#creamos una carpeta para cada modelo
tensorboardCNN = TensorBoard(log_dir='logs/cnn')
#con fit le mando la X(imagenes) y las Y(etiquetas) y el lote, dejando un % para pruebas
modeloCNN.fit(X,y,batch_size=32,
                validation_split = 0.15,
                epochs = 100,
                callbacks=[tensorboardCNN])# por cada epoca que recorra y al final guarda en un archivo 


In [ ]:
#creamos una carpeta para cada modelo
tensorboardCNN2 = TensorBoard(log_dir='logs/cnn2')
#con fit le mando la X(imagenes) y las Y(etiquetas) y el lote, dejando un % para pruebas
modeloCNN2.fit(X,y,batch_size=32,
                validation_split = 0.15,
                epochs = 100,
                callbacks=[tensorboardCNN2])# por cada epoca que recorra y al final guarda en un archivo 

In [26]:
#visualizamos los datos en tensorboard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
#lanzamos la caperta donde se 
%tensorboard --logdir logs

In [31]:
!kill 3106

In [ ]:
plt.figure(figsize=(20,8))
for i in range(10):
  plt.subplot(2,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(X[i].reshape(100,100), cmap='gray')

In [ ]:
#aumento de datos en el entrenamiento
#Realizar el aumento de datos con varias transformaciones. Al final, graficar 10 como ejemplo
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range = 30, #rotacion aleatoria de las imagenes
    width_shift_range=0.2,#mover de los lados
    height_shift_range=0.2,#mover arriba y abajo
    shear_range=15,#inclinacion de imagen
    zoom_range=[0.7, 1.4], # que tan pequeño el acercamiento de la imagen
    horizontal_flip=True,# roataciones 
    vertical_flip=True
)

datagen.fit(X) #para darle el arreglo de imagenes a la funcion fit
#esto espara que el generador entienda todo lo referente al tamaño de las imagenes y que pueda ver como hara las transformaciones cuando se lo pida

#ahora para las imagenes transformadas hay muchas funciones
plt.figure(figsize=(20,8))
#con el flow le damos las imagenes, en este caso 10, y q las de en el orden original
#y como flow es un iterador lo ponemos en un for

for imagen, etiqueta in datagen.flow(X, y, batch_size=10, shuffle=False):
  for i in range(10):
    plt.subplot(2,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X[i].reshape(100,100), cmap='gray')
  break

In [38]:
modeloDenso_AD = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modeloCNN_AD = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modeloCNN2_AD = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [39]:
modeloDenso_AD.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN_AD.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN2_AD.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [48]:
#para el aumento de datos si toca separar datos de entramientos y de pruebas
len(X) * .85 #19700
len(X) - 19700 #3562

#definimos dos variables
X_entrenamiento = X[:19700] #desde el inicion hasta el valor 19700 -->85% de entramiento
X_validacion = X[19700:] #desde el punto donde se quedo hasta el final del arreglo ---> 15% de pruebas

#ahora para las etiquetas el mismo procedimiento que con las X
#definimos dos variables
y_entrenamiento = y[:19700] #desde el inicion hasta el valor 19700 -->85% de entramiento
y_validacion = y[19700:] #desde el punto donde se quedo hasta el final del arreglo ---> 15% de pruebas

In [49]:
#preeparar el generador de datos transformados para el entrenamiendo de datos
#Usar la funcion flow del generador para crear un iterador que podamos enviar como entrenamiento a la funcion FIT del modelo
data_gen_entrenamiento = datagen.flow(X_entrenamiento, y_entrenamiento, batch_size=32)

In [50]:
import gc
gc.collect()

46068

In [ ]:
tensorboardCNN_AD = TensorBoard(log_dir='logs-new/cnn_AD')

modeloCNN_AD.fit(
    data_gen_entrenamiento,
    epochs=150, batch_size=32,
    validation_data=(X_validacion, y_validacion),
    steps_per_epoch=int(np.ceil(len(X_entrenamiento) / float(32))),
    validation_steps=int(np.ceil(len(X_validacion) / float(32))),
    callbacks=[tensorboardCNN_AD]
)

In [53]:
modeloCNN_AD.save('perros-gatos-cnn-ad.h5')

In [ ]:
!pip install tensorflowjs

In [55]:
!mkdir carpeta_salida